In [1]:
!apt-get -qq install -y ffmpeg  # Colab에서만 필요

In [2]:
!pip install -q git+https://github.com/openai/whisper.git yt-dlp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.6 MB/s eta 0:00:00


In [10]:
import whisper
import yt_dlp
import os
import torch

def download_audio_from_youtube(url, output_filename="downloaded_audio.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_filename.split('.')[0],
        'quiet': False,
    }

    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"기존 '{output_filename}' 파일을 삭제했습니다.")

    print(f"\n'{url}' 영상에서 오디오 다운로드를 시작합니다...")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    print(f"\n✅ 오디오 다운로드 완료! -> Colab 왼편 파일 탐색기에서 '{output_filename}' 확인 가능")
    return output_filename

def transcribe_audio_with_whisper(audio_path):
    print("\nWhisper 모델을 로딩하고 음성 인식을 시작합니다...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"사용 장치: {device}")

    try:
        model = whisper.load_model("base", device=device)
        result = model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        return f"[오류] 음성 인식에 실패했습니다: {e}"

# 1. 텍스트로 변환하고 싶은 유튜브 영상 URL
youtube_url = "https://www.youtube.com/watch?v=4--IoHXmj2U"
video_id = youtube_url.split('v=')[-1]

# 2. 오디오 다운로드 실행
audio_file = download_audio_from_youtube(youtube_url)

# 3. 다운로드된 오디오 파일로 음성 인식 실행
if audio_file:
    full_text = transcribe_audio_with_whisper(audio_file)
    print("\n=====  최종 음성 인식 결과 =====")
    print(full_text)

    # 4. 변환된 텍스트를 txt 파일로 저장
    output_txt_filename = f"{video_id}_transcript.txt"
    with open(output_txt_filename, "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"\n✅ 텍스트가 '{output_txt_filename}' 파일로 저장되었습니다.")

기존 'downloaded_audio.mp3' 파일을 삭제했습니다.

'https://www.youtube.com/watch?v=4--IoHXmj2U' 영상에서 오디오 다운로드를 시작합니다...
[youtube] Extracting URL: https://www.youtube.com/watch?v=4--IoHXmj2U
[youtube] 4--IoHXmj2U: Downloading webpage
[youtube] 4--IoHXmj2U: Downloading tv simply player API JSON
[youtube] 4--IoHXmj2U: Downloading tv client config
[youtube] 4--IoHXmj2U: Downloading tv player API JSON
[info] 4--IoHXmj2U: Downloading 1 format(s): 251
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: downloaded_audio
[download] 100% of   40.70MiB in 00:00:07 at 5.30MiB/s   
[ExtractAudio] Destination: downloaded_audio.mp3
Deleting original file downloaded_audio (pass -k to keep)

✅ 오디오 다운로드 완료! -> Colab 왼편 파일 탐색기에서 'downloaded_audio.mp3' 확인 가능

Whisper 모델을 로딩하고 음성 인식을 시작합니다...
사용 장치: cuda

=====  최종 음성 인식 결과 =====
 Thank you all for coming and welcome to our music event today. You know, Apple's leading the digital music revolution, but to us at its core, it's all about 